In [1]:
from pcntoolkit.util.hbr_utils import *
import scipy

import os
import pickle
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import pcntoolkit as ptk
from itertools import product
from functools import reduce
from pcntoolkit.model.SHASH import SHASHb, SHASH, SHASHo
from pcntoolkit.util.utils import scaler
from scipy.stats import gaussian_kde
import pymc as pm
import scipy.special as spp
import arviz as av
from scipy.stats import skew, kurtosis
import seaborn as sns
import time
sns.set(style='darkgrid')

In [2]:
projdir = '/project_cephfs/3022017.02/projects/stijdboe/make_results/10_folds_results/'
data_dir = '/project_cephfs/3022017.02/projects/stijdboe/Data'
folds_dir = os.path.join(data_dir,'10_folds')
python_path = '/project_cephfs/3022017.02/projects/stijdboe/stijn2/bin/python3.10'
normative_path = '/project_cephfs/3022017.02/projects/stijdboe/PCNtoolkit/pcntoolkit/normative.py'

In [3]:
# For each fold
n_folds = 10
for i_fold in range(n_folds):
    
    # Load the train and test data
    fold_data_dir = f'/project_cephfs/3022017.02/projects/stijdboe/Data/10_folds_sexcov/fold_{i_fold}'
    with open(os.path.join(fold_data_dir, 'X_train.pkl'),'rb') as file:
              X_train = pickle.load(file)
    with open(os.path.join(fold_data_dir, 'Y_train.pkl'),'rb') as file:
              Y_train = pickle.load(file)
    with open(os.path.join(fold_data_dir, 'Z_train.pkl'),'rb') as file:
              Z_train = pickle.load(file)
    with open(os.path.join(fold_data_dir, 'X_test.pkl'),'rb') as file:
              X_test = pickle.load(file)
    with open(os.path.join(fold_data_dir, 'Y_test.pkl'),'rb') as file:
              Y_test = pickle.load(file)
    with open(os.path.join(fold_data_dir, 'Z_test.pkl'),'rb') as file:
              Z_test = pickle.load(file)

    # For each feature
    features = ['Right-Cerebellum-White-Matter',
                'EstimatedTotalIntraCranialVol',
                'Right-Lateral-Ventricle',
                'WM-hypointensities',
                'rh_S_interm_prim-Jensen_thickness', 
                'Brain-Stem',
                'log_WM-hypointensities']
    for i_f, feature in enumerate(features):    
        
        # Load that particular data
        this_X_train = X_train.to_numpy()
        this_Y_train = Y_train[feature].to_numpy()
        this_X_test = X_test.to_numpy()
        this_Y_test = Y_test[feature].to_numpy()
        this_Z_test = Z_test.to_numpy()
        
        # Apply in- and outscale to train and test (use util.scaler)
        inscaler = scaler('standardize')
        outscaler = scaler('standardize')
        inscaler.fit(this_X_train)
        outscaler.fit(this_Y_train)
        this_X_test = inscaler.transform(this_X_test)
        this_Y_test = outscaler.transform(this_Y_test)
        
        # For each model type:
        for model_type in ['SHASHo', 'SHASHb_1','SHASHb_2','Normal']:
        # for model_type in ['SHASHo']:
        # for model_type in ['SHASHb_2']:

        # for model_type in ['SHASHb_1','Normal']:

            # Load the model
            fold_dir = f'fold_{i_fold}_{model_type}'
            batch_dir = f'batch_{i_f+1}'
            nm_name = f'NM_0_0_estimate.pkl'
            model_path = os.path.join(projdir, fold_dir, batch_dir, 'Models', nm_name)
            if os.path.exists(model_path):
                with open(model_path, 'rb') as file:
                    norm_hbr_model = pickle.load(file)
                # Find the MCMC z-scores
                print(f"Computing zscores for {nm_name}")
                start = time.time()
                MCMC_z_scores = norm_hbr_model.get_mcmc_zscores(np.squeeze(this_X_test), this_Y_test[:,None], this_Z_test)
                a = norm_hbr_model.hbr.idata
                end = time.time()
                print(f"Done computing zscores in {end-start} seconds")
                
                mcmc_zscoresdir = os.path.join(projdir, fold_dir, batch_dir, 'MCMC_zscores')
                if not os.path.exists(mcmc_zscoresdir):
                    os.mkdir(mcmc_zscoresdir)
                with open(os.path.join(mcmc_zscoresdir, f'MCMC_zscores_{i_f}_{model_type}_fold{i_fold}.pkl'),'wb') as file:
                    pickle.dump(MCMC_z_scores, file)


Computing zscores for NM_0_0_estimate.pkl
SHASHo


Sampling: [y_like]


Done computing zscores in 12.251747131347656 seconds
Computing zscores for NM_0_0_estimate.pkl
SHASHb


Sampling: [y_like]


Done computing zscores in 6.144245386123657 seconds
Computing zscores for NM_0_0_estimate.pkl
SHASHb


...

Done computing zscores in 3.8815183639526367 seconds
Computing zscores for NM_0_0_estimate.pkl
SHASHb


Sampling: [y_like]


Done computing zscores in 33.941346645355225 seconds
Computing zscores for NM_0_0_estimate.pkl
Normal


Sampling: [y_like]


Done computing zscores in 1.6523911952972412 seconds
